In [17]:
import cv2
import log_reg
import numpy as np
from glob import glob
from matplotlib.pyplot import imread, imshow
from util import read_X_Y, get_data_col, filter_data_by_colval
from keras.models import Sequential 
from keras.layers import Dense, Activation, Reshape,Input
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
#TensorBoard
from keras.callbacks import TensorBoard
from time import time
import tensorflow as tf


import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

In [2]:
file_names = sorted(glob('all-mias/*.pgm'))

In [3]:
headers = ['file_name', 'character', 'class', 'severity', 'x', 'y', 'radius']
filtered_X, Y_headers,Y_names = read_X_Y('dataset/data.txt',headers)

In [4]:
image_stack = np.array([imread(file_name) for file_name in filtered_X])

In [5]:
def produce_y_mask(Y,Y_names):
    for i in range(len(Y)):
        image = np.zeros((1024,1024,1))
        cv2.circle(image, (int(Y[i][0]),1024-int(Y[i][1])),int(Y[i][2]),(255),-1)
        name = 'mias_y_masked/' + Y_names[i] + '.png'
        cv2.imwrite(name, image)

In [6]:
# print(Y_headers)
produce_y_mask(Y_headers,Y_names)

In [5]:
Y_mask_filenames = ['mias_y_masked/' + file_name + '.png' for file_name in Y_names]
Y = np.array([imread(file_name) for file_name in Y_mask_filenames])

In [6]:
Y = np.array(Y)
Y = np.squeeze(Y)
m,l,h = image_stack.shape

print(image_stack.shape)
X = image_stack.reshape((m,l,h,1))
Y = Y.reshape((m,l*h,1))
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,train_size=0.6,random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_test,Y_test,train_size=0.5,random_state=42)

# X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
# X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
# X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)

print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)
print(X_test.shape)
print(Y_test.shape)

# Y_train = Y_train.reshape((m,l*h,1))



(119, 1024, 1024)


/home/shared/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


(71, 1024, 1024, 1)
(71, 1048576, 1)
(24, 1024, 1024, 1)
(24, 1048576, 1)
(24, 1024, 1024, 1)
(24, 1048576, 1)


In [7]:
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)
Y_val = to_categorical(Y_val)

In [8]:
#Taken from https://stackoverflow.com/questions/45947351/how-to-use-tensorflow-metrics-in-keras

def as_keras_metric(method):
    import functools
    from keras import backend as K
    import tensorflow as tf
    @functools.wraps(method)
    def wrapper(self, args, **kwargs):
        """ Wrapper for turning tensorflow metrics into keras metrics """
        value, update_op = method(self, args, **kwargs)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
            value = tf.identity(value)
        return value
    return wrapper

In [9]:
#Taken from https://stackoverflow.com/questions/45947351/how-to-use-tensorflow-metrics-in-keras

@as_keras_metric
def mean_iou(y_true, y_pred, num_classes=2):
    return tf.metrics.mean_iou(y_true, y_pred, num_classes)

In [10]:
model = Sequential()
model.add(Conv2D(16, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=(l,h,1)))
model.add(Conv2D(8, kernel_size=(5, 5),
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(50, 50)))
model.add(Dropout(0.25))
model.add(Conv2D(4, kernel_size=(5, 5),
                 activation='relu'))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1048576*2))
model.add(Reshape((1048576, 2)))
model.add(Activation('softmax'))

model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=[mean_iou])
# model.summary()
# print(X_train.shape)
# print(Y_train.shape)
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
model.fit(X_train,Y_train,epochs=20, validation_data=(X_val, Y_val), batch_size=1,verbose=1, callbacks=[tensorboard])

Instructions for updating:
Use the retry module or similar alternatives.
Train on 71 samples, validate on 24 samples
Epoch 1/20
71/71 [==============================] - 26s 372ms/step - loss: 0.1416 - mean_iou: 0.6989 - val_loss: 0.2349 - val_mean_iou: 0.8981
Epoch 2/20
71/71 [==============================] - 14s 191ms/step - loss: 0.1099 - mean_iou: 0.9283 - val_loss: 0.2353 - val_mean_iou: 0.9427
Epoch 3/20
71/71 [==============================] - 14s 193ms/step - loss: 0.1101 - mean_iou: 0.9509 - val_loss: 0.2353 - val_mean_iou: 0.9564
Epoch 4/20
71/71 [==============================] - 14s 192ms/step - loss: 0.1101 - mean_iou: 0.9607 - val_loss: 0.2353 - val_mean_iou: 0.9631
Epoch 5/20
71/71 [==============================] - 14s 192ms/step - loss: 0.1101 - mean_iou: 0.9656 - val_loss: 0.2353 - val_mean_iou: 0.9671
Epoch 6/20
71/71 [==============================] - 14s 194ms/step - loss: 0.1101 - mean_iou: 0.9685 - val_loss: 0.2353 - val_mean_iou: 0.9697
Epoch 7/20
71/71 [=======

In [21]:
model = Sequential()
model.add(Conv2D(16, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=(l,h,1)))
model.add(Conv2D(8, kernel_size=(5, 5),
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(50, 50)))
model.add(Dropout(0.25))
model.add(Conv2D(4, kernel_size=(5, 5),
                 activation='relu'))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1048576*2))
model.add(Reshape((1048576, 2)))
model.add(Activation('softmax'))

model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=[mean_iou])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_87 (Conv2D)           (None, 1020, 1020, 16)    416       
_________________________________________________________________
conv2d_88 (Conv2D)           (None, 1016, 1016, 8)     3208      
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 20, 20, 8)         0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 20, 20, 8)         0         
_________________________________________________________________
conv2d_89 (Conv2D)           (None, 16, 16, 4)         804       
_________________________________________________________________
flatten_2 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                10250     
__________

In [27]:
inputs = Input((l,h,1))
conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
drop4 = Dropout(0.5)(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
drop5 = Dropout(0.5)(conv5)

up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
merge6 = concatenate([drop4,up6], axis = 3)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
merge7 = concatenate([conv3,up7], axis = 3)
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
merge8 = concatenate([conv2,up8], axis = 3)
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
merge9 = concatenate([conv1,up9], axis = 3)
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
reshape2d = Reshape((1048576,2))(conv9)
foutput = Activation('softmax')(reshape2d)
# model.add(Activation('softmax'))

# conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)
# foutput = Reshape((1048576,1))(conv10)

model = Model(input = inputs, output = foutput)
model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics=[mean_iou])
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
model.summary()
#model.fit(X_train,Y_train,epochs=20, validation_data=(X_val, Y_val), batch_size=1,verbose=1, callbacks=[tensorboard])

/home/shared/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ac...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 1024, 1024, 1 0                                            
__________________________________________________________________________________________________
conv2d_186 (Conv2D)             (None, 1024, 1024, 6 640         input_9[0][0]                    
__________________________________________________________________________________________________
conv2d_187 (Conv2D)             (None, 1024, 1024, 6 36928       conv2d_186[0][0]                 
__________________________________________________________________________________________________
max_pooling2d_35 (MaxPooling2D) (None, 512, 512, 64) 0           conv2d_187[0][0]                 
__________________________________________________________________________________________________
conv2d_188

In [ ]:
model.fit(X_train,Y_train,epochs=20, validation_data=(X_val, Y_val), batch_size=1,verbose=1, callbacks=[tensorboard])

Train on 71 samples, validate on 24 samples
Epoch 1/20
71/71 [==============================] - 271s 4s/step - loss: 0.6937 - mean_iou: 0.2465 - val_loss: 0.6931 - val_mean_iou: 0.2500
Epoch 2/20
71/71 [==============================] - 244s 3s/step - loss: 0.6931 - mean_iou: 0.2500 - val_loss: 0.6931 - val_mean_iou: 0.2500
Epoch 3/20
71/71 [==============================] - 244s 3s/step - loss: 0.6931 - mean_iou: 0.2500 - val_loss: 0.6931 - val_mean_iou: 0.2500
Epoch 4/20
71/71 [==============================] - 244s 3s/step - loss: 0.6931 - mean_iou: 0.2500 - val_loss: 0.6931 - val_mean_iou: 0.2500
Epoch 5/20
71/71 [==============================] - 244s 3s/step - loss: 0.6931 - mean_iou: 0.2500 - val_loss: 0.6931 - val_mean_iou: 0.2500
Epoch 6/20
71/71 [==============================] - 244s 3s/step - loss: 0.6931 - mean_iou: 0.2500 - val_loss: 0.6931 - val_mean_iou: 0.2500
Epoch 7/20
71/71 [==============================] - 244s 3s/step - loss: 0.6931 - mean_iou: 0.2500 - val_loss:

In [ ]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=(l,h,1)))
model.add(Conv2D(128, kernel_size=(5, 5),
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(100, 100)))
model.add(Dropout(0.25))
model.add(Conv2D(128, kernel_size=(5, 5),
                 activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1048576*2))
model.add(Reshape((1048576, 2)))
model.add(Activation('softmax'))

model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=[mean_iou])
# model.summary()
# print(X_train.shape)
# print(Y_train.shape)
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
model.fit(X_train,Y_train,epochs=20, validation_data=(X_val, Y_val), batch_size=1,verbose=1, callbacks=[tensorboard])

Train on 71 samples, validate on 24 samples
Epoch 1/20
 2/71 [..............................] - ETA: 1:07:50 - loss: 0.6889 - mean_iou: 0.1250    

In [7]:
#General model seems to need an extremely large output!
model = Sequential([
    Dense(128, input_shape=(l*h,)),
    Activation('relu'),
    Dense(64),
    Activation('relu'),
    Dense(32),
    Activation('relu'),
    Dense(32),
    Activation('relu'),
    Dense(3),
    Activation('relu'),
])
model.compile(optimizer='Adam',
              loss='categorical_crossentropy')
model.fit(X_train,Y_train,epochs=20, validation_data=(X_val, Y_val), batch_size=32)

Train on 71 samples, validate on 24 samples
Epoch 1/20
71/71 [==============================] - 26s 366ms/step - loss: 16666.3377 - val_loss: 11683.5654
Epoch 2/20
71/71 [==============================] - 13s 185ms/step - loss: 7853.4531 - val_loss: 453.2060
Epoch 3/20
71/71 [==============================] - 11s 157ms/step - loss: 385.5853 - val_loss: 344.5707
Epoch 4/20
71/71 [==============================] - 8s 118ms/step - loss: 350.8421 - val_loss: 344.5707
Epoch 5/20
71/71 [==============================] - 10s 134ms/step - loss: 350.8421 - val_loss: 344.5707
Epoch 6/20
71/71 [==============================] - 13s 183ms/step - loss: 350.8421 - val_loss: 344.5707
Epoch 7/20
71/71 [==============================] - 10s 136ms/step - loss: 350.8421 - val_loss: 344.5707
Epoch 8/20
71/71 [==============================] - 9s 132ms/step - loss: 350.8421 - val_loss: 344.5707
Epoch 9/20
71/71 [==============================] - 10s 137ms/step - loss: 350.8421 - val_loss: 344.5707
Epoch 10

In [8]:
y_pred_val = model.predict(X_train)

In [9]:
print(y_pred_val)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [28]:
print(Y_val)

[[448. 480.  95.]
 [492. 473. 131.]
 [600. 514.  67.]
 [518. 191.  39.]
 [356. 945.  72.]
 [647. 503.  87.]
 [523. 482.  29.]
 [547. 520.  45.]
 [366. 620.  33.]
 [525. 425.  33.]
 [489. 480.  82.]
 [415. 460.  38.]
 [326. 607. 174.]
 [352. 624. 114.]
 [462. 406.  44.]
 [603. 538.  44.]
 [522. 553.  17.]
 [519. 362.  54.]
 [592. 670.  33.]
 [493. 125.  49.]
 [612. 297.  34.]
 [191. 549.  23.]
 [400. 484.  37.]
 [653. 477.  49.]]


In [ ]:
model2 = Sequential([
    Dense(1024, input_shape=(l*h,)),
    Activation('relu'),
    Dense(512),
    Activation('relu'),
    Dense(256),
    Activation('relu'),
    Dense(64),
    Activation('relu'),
    Dense(3),
    Activation('relu'),
])
model2.compile(optimizer='Adam',
              loss='mean_squared_error')
model2.fit(X_train,Y_train,epochs=20, validation_data=(X_val, Y_val), batch_size=32)

Train on 71 samples, validate on 24 samples
Epoch 1/20


In [18]:
model2 = Sequential([
    Dense(128, input_shape=(l*h,)),
    Activation('relu'),
    Dense(64),
    Activation('relu'),
    Dense(32),
    Activation('relu'),
    Dense(16),
    Activation('relu'),
    Dense(3),
    Activation('relu'),
])
model2.compile(optimizer='Adam',
              loss='mean_squared_error')
# print(X_train[0].shape)
model2.fit(,epochs=5, batch_size=32)

(1048576,)


ValueError: Error when checking target: expected activation_55 to have shape (3,) but got array with shape (1,)

In [9]:
image = Image.new('1', (1024, 1024)) #create new image, 10x10 pixels, 1 bit per pixel
print(Y_train[0])

[492. 434.  87.]


In [ ]:
draw = ImageDraw.Draw(image)
draw.ellipse((2, 2, 8, 8), outline ='white')
print list(image.getdata())